# Handling Missing Data with Pandas

This section from the code camp level covers conceptual theory and cleaning of data in python with numpy and pandas. Pandas specifically borrows all of the capabilities from numpy selection, and adds a number of other methods for handling those items.

We can very rapidly identify the missing data; however, there are other issues that can occur within a specific domain, for example, age might have an entry of 200, we know this is invalid inherently within that domain, but in another domain you may not know all this information.

As such, it is important that when you are working on an analysis project that you gain some understanding of the domain, this could be very much something like knowledge engineering in an expert system.

## Hands on

In [2]:
import numpy as np
import pandas as pd

## Pandas utility functions

These are all similar to numpy for identifying null values.

In [7]:
pd.isnull(np.nan)

True

In [8]:
pd.isnull(None)

True

In [9]:
pd.isna(np.nan)

True

In [10]:
pd.isna(None)

True

Similarly for ``pd.notnull(N)`` for identifying non-null values.

These work on whole series, and thus dataframes

In [13]:
pd.isnull(pd.DataFrame({
        "Column A" : [13, np.nan, 7],
        "Column B" : [np.nan, 100, 50]
}))

,Column A,Column B
0,False,True
1,True,False
2,False,False


## Filtering null values

In [14]:
s = pd.Series([1,2,3,np.nan, np.nan, 4])

In [15]:
pd.notnull(s)

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [17]:
pd.notnull(s).sum() # Count not null

4

In [18]:
pd.isnull(s).sum() # Count null

2

We can opt to drop those with null values

In [19]:
s.dropna()

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

These methods are immutable, and as such, we generate a new dataframe post these actions, which we can assign to a different variable, whilst still maintaining the posterity of the original data. These methods also work on dataframes.

In [21]:
df = pd.DataFrame({
    "Column A": [1, np.nan, 30, np.nan],
    "Column B": [2, 8, 31, np.nan],
    "Column C": [np.nan, 9, 32, 100],
    "Column D": [5, 8, 34, 110]
})

In [22]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


We can deal with this by dropping rows or columns that have null values; however, this can be very harsh on your data.

In [24]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In [25]:
df.dropna(axis=1)

,Column D
0,5
1,8
2,34
3,110


Here we lose a massive amount of information that might be useful elsewise. We can set a threshold for this, to maintain some additional data.

In [26]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


We can choose (perhaps by some informed process) to fill the null data with some other information, that could be a default value, a mean or median value, or forward or backward filling.

In [27]:
df.fillna(0) # Fill null with 0

,Column A,Column B,Column C,Column D
0,1.0,2.0,0.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,0.0,100.0,110


In [32]:
df.fillna(df.mean()) # Fills null with mean per column

,Column A,Column B,Column C,Column D
0,1.0,2.000000,47.0,5
1,15.5,8.000000,9.0,8
2,30.0,31.000000,32.0,34
3,15.5,13.666667,100.0,110


In [33]:
df.fillna(method='ffill')

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [34]:
df.fillna(method='bfill')

,Column A,Column B,Column C,Column D
0,1.0,2.0,9.0,5
1,30.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


You must be careful with forward or backward fill, as we see above, if you have end or beginning elements which are null, you will end up with null values still.

## Invalid values

In [39]:
df = pd.DataFrame({
    "Age": [20, 30, 50, 290, 36],
    "Sex": ['M', 'F', 'F', 'D', '?']
})
df

,Age,Sex
0,20,M
1,30,F
2,50,F
3,290,D
4,36,?


Whilst there are no missing values, there are invalid ones, for example, there are only two biological sexes, male (M) and female(F). We can use replacement to change these should we want.

In [40]:
df.replace({
    'Sex':{
        'D':'F',
        '?':'M'
    }
})

,Age,Sex
0,20,M
1,30,F
2,50,F
3,290,F
4,36,M


With regards to the age, we can likely assume this is an extra 0 by a typo. We could use a function to remove these.

In [43]:
df.loc[df['Age']>100, 'Age'] = df.loc[df['Age']> 100, 'Age']/10

In [44]:
df

,Age,Sex
0,20,M
1,30,F
2,50,F
3,29,D
4,36,?


## Duplicates

From the online example at: https://github.com/ine-rmotr-curriculum/data-cleaning-rmotr-freecodecamp/blob/master/3%20-%20Cleaning%20Not%20Null%20Values.ipynb

"""Checking duplicate values is extremely simple. It'll behave differently between Series and DataFrames. Let's start with Series. As an example, let's say we're throwing a fancy party and we're inviting Ambassadors from Europe. But can only invite one ambassador per country. This is our original list, and as you can see, both the UK and Germany have duplicated ambassadors:"""

In [45]:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])

In [46]:
ambassadors

Gérard Araud                  France
Kim Darroch           United Kingdom
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
Peter Ammon                  Germany
Klaus Scharioth              Germany
dtype: object

We can check if there are duplicates in the data by using the ```duplicated``` function

In [47]:
ambassadors.duplicated()

Gérard Araud          False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

This goes top down, it can also be done bottom up by ```.duplicated(keep='last')```. Or, we can exclude all duplicated entried by:

In [48]:
ambassadors.duplicated(keep=False)

Gérard Araud          False
Kim Darroch            True
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

All of these methods work with drop duplicates in the same way:

In [49]:
ambassadors.drop_duplicates()

Gérard Araud                  France
Kim Darroch           United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
dtype: object

In [50]:
ambassadors.drop_duplicates(keep='last')

Gérard Araud                  France
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Klaus Scharioth              Germany
dtype: object

In [51]:
ambassadors.drop_duplicates(keep=False)

Gérard Araud          France
Armando Varricchio     Italy
dtype: object

## Text entries

One can also clean text, and this uses many of the same methods used in base python.

## Splitting columns

In [61]:
df = pd.DataFrame({
    'Data': [
        '1987_M_US _1',
        '1990?_M_UK_1',
        '1992_F_US_2',
        '1970?_M_   IT_1',
        '1985_F_I  T_2'
]})

In [62]:
df

,Data
0,1987_M_US _1
1,1990?_M_UK_1
2,1992_F_US_2
3,1970?_M_ IT_1
4,1985_F_I T_2


In this case, if we know the data structure, we can split in the standard split method from python by identifying the delimiter.

In [63]:
df = df['Data'].str.split('_', expand=True)
df

,0,1,2,3
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


We can then name the columns by the data we know.

In [65]:
df.columns = ['Year', 'Sex', 'Country', 'Num Children']
df

,Year,Sex,Country,Num Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [66]:
df

,Year,Sex,Country,Num Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [69]:
df['Country'].str.replace(' ', '')

0    US
1    UK
2    US
3    IT
4    IT
Name: Country, dtype: object

In [70]:
df

,Year,Sex,Country,Num Children
0,1987,M,US,1
1,1990?,M,UK,1
2,1992,F,US,2
3,1970?,M,IT,1
4,1985,F,I T,2


In [72]:
df['Year'].str.replace(r'(?P<year>\d{4})\?', lambda m: m.group('year'), regex=True)

0    1987
1    1990
2    1992
3    1970
4    1985
Name: Year, dtype: object